In [1]:
import time
# pour la gestion du GPU, j'utilise thinc.api finalement au lieu de spacy

# pour un ordi avec GPU :

import GPUtil
import torch
!set CUDA_PATH="/usr/local/cuda-12"
GPUtil.getAvailable()
use_cuda = torch.cuda.is_available()
print("is cuda available? ", use_cuda)

is cuda available?  True


In [2]:
import pandas as pd
import spacy

In [3]:
def print_full(df):
    pd.set_option('display.max_colwidth', 2000)
    # pd.set_option('display.width', 2000)

    print(df)
    # pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 2000)
# pd.set_option('display.float_format', '{:20,.2f}'.format)
# pd.set_option('display.max_colwidth', None)
# print(x)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
# pd.reset_option('display.width')
# pd.reset_option('display.float_format')
# pd.reset_option('display.max_colwidth')

In [4]:
df = pd.read_csv("df_CC_cleaned.csv", sep = ",", encoding = "utf-8", dtype= str)

In [6]:
# vérifier l'état des données

if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    print(df[df.isnull().T.any()])
else :
    print("il n'y a pas de NaN")

print("quels types pour chaque colonne?\n")    
for column in df.columns:
    print(pd.api.types.infer_dtype(df[column]))
print("y a-t-il des types mixés dans les données?\n")
print(df[column][df[column].apply(lambda x: isinstance(x, type))])
print(df._is_mixed_type)
print(df.dtypes.nunique()>1)

il n'y a pas de NaN
quels types pour chaque colonne?

string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
y a-t-il des types mixés dans les données?

Series([], Name: all_lemmas_without_stopwords, dtype: object)
False
False


In [11]:
if df.isnull().values.any() :
    print("y a t il des nan? ->",df.isnull().values.any())
    print("combien y a t il de nan? ->",df.isnull().values.sum())
    print("où sont les null? ->\n",df.isnull().sum())
    df[df.isnull().T.any()]
else :
    print("y a t il des nan? ->",df.isnull().values.any())

y a t il des nan? -> False


In [12]:
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
set_gpu_allocator("pytorch")
require_gpu() 
set_active_gpu(0)
if prefer_gpu():
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    use_pytorch_for_gpu_memory()

    print("GPU Usage")
    GPUtil.showUtilization()


Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  3% |  4% |


In [6]:
# from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
# set_gpu_allocator("pytorch")
def lemmatize(doc, subset, stopwords = False):
    '''
    Performs lemmatization of input documents, with batching to avoid a ram overload
    Args:
    - docs: one column of a dataframe
    Output:
    - write csvs the size of the subset
    '''

    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.

    t0 = time.time()
    
    # count the number of batches
    len_data = doc.shape[0]
    print("subset : ", subset)
    nb_batch = int(len_data/subset)+ (len_data % subset > 0)
    print("nb_batch : ", nb_batch)
    # size of last bacth
    size_last_batch = len_data%subset
    print("size_last_batch : ", size_last_batch)
    first_idx = 0
    last_idx = 0
        
    final_lemmas_df = pd.DataFrame()
    final_postag_df = pd.DataFrame()


    # creating one batch (a slice of the df)
    for batch_nb in range(nb_batch) :
        t1 = time.time()
        lemma_text_list = []
        lemma_pos_list = []
        print(f"starting batch n° {batch_nb} , on {nb_batch} batches")
        # range() commence à 0 et exclut nb_batch : donc on utilise nb_batch -1 pout détecter le dernier batch
        # si c'est le dernier batch, on ajoute la taille du batch final au lieu de la taille prévue
        if batch_nb == nb_batch -1 :
            first_idx = last_idx
            last_idx += size_last_batch
        else :
            first_idx = last_idx
            last_idx += subset
        print(f"first_index : {first_idx} and last_idx : {last_idx}")
        batch = doc[first_idx : last_idx]
        
        # spacy tools go through the batch
        for line in nlp.pipe(batch):
            if stopwords is True:
                lemma_pos_list.append([x.pos_ for x in line if x.pos_ in {"VERB", "ADJ", "ADV", "NOUN"}])
                lemma_text_list.append([x.lemma_ for x in line if x.pos_ in {"VERB", "ADJ", "ADV","NOUN"}])

            else :
                lemma_pos_list.append([x.pos_ for x in line])
                lemma_text_list.append([x.lemma_ for x in line])

        print("GPU Usage")
        GPUtil.showUtilization()

        # writing the output lemmas and postag in one file each
        # n contributions per file, n the subset

        df_lemma = pd.DataFrame([lemma_text_list])
        df_postag = pd.DataFrame([lemma_pos_list])


        final_lemmas_df = pd.concat([final_lemmas_df,df_lemma.T])
        final_postag_df = pd.concat([final_postag_df,df_postag.T])

        # deleting the lists for clearing, to force python garbage collector to *actually* collect
        del df_lemma
        del df_postag

        del lemma_text_list
        del lemma_pos_list
    
        t2 = time.time()
        print(f"end of batch n°: {batch_nb}, time: {t2-t1}")
        
    t3 = time.time()
    print("Total time: {}".format(t3-t0))
    return(final_lemmas_df, final_postag_df)


In [14]:
# lemmatize and postag on clean text
final_lemmas_df, final_postag_df = lemmatize(df["clean"], 128, stopwords = False)

subset :  128
nb_batch :  1759
size_last_batch :  21
starting batch n° 0 , on 1759 batches
first_index : 0 and last_idx : 128
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 11% |
end of batch n°: 0, time: 8.02172589302063
starting batch n° 1 , on 1759 batches
first_index : 128 and last_idx : 256
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
end of batch n°: 1, time: 3.6409945487976074
starting batch n° 2 , on 1759 batches
first_index : 256 and last_idx : 384
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
end of batch n°: 2, time: 2.8092033863067627
starting batch n° 3 , on 1759 batches
first_index : 384 and last_idx : 512
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
end of batch n°: 3, time: 2.5040271282196045
starting batch n° 4 , on 1759 batches
first_index : 512 and last_idx : 640
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 13% |
end of batch n°: 4, time: 2.3393032550811768
starting batch n

In [15]:
# adding a column to the df
# renumber the index column (not accurate bc origin from each concatenation)

forconcat_lemmas_df = final_lemmas_df.reset_index()
forconcat_postag_df = final_postag_df.reset_index()

# drop the secoond index column

forconcat_lemmas_df = forconcat_lemmas_df.drop(labels = "index", axis=1)
forconcat_postag_df = forconcat_postag_df.drop(labels = "index", axis=1)

# name the columns 

forconcat_lemmas_df.rename(columns = {0:"lemmas"}, inplace=True)
forconcat_postag_df.rename(columns = {0:"postags"}, inplace=True)

# concatenate the columns to the side of the original csv

df = pd.concat([df,forconcat_lemmas_df], axis = 1)
df = pd.concat([df,forconcat_postag_df], axis = 1)

In [16]:
# exporter le dataframe dans un csv

df.to_csv("df_CC_lemmas_postag.csv", index = False)

In [7]:
# lemmatize and postagging on only verb, noun, adj, adv
model_lemmas_df, model_postag_df = lemmatize(df["clean"], 128, stopwords = True)

subset :  128
nb_batch :  1759
size_last_batch :  21
starting batch n° 0 , on 1759 batches
first_index : 0 and last_idx : 128
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
end of batch n°: 0, time: 1.6178522109985352
starting batch n° 1 , on 1759 batches
first_index : 128 and last_idx : 256
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
end of batch n°: 1, time: 1.460132360458374
starting batch n° 2 , on 1759 batches
first_index : 256 and last_idx : 384
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
end of batch n°: 2, time: 1.1061100959777832
starting batch n° 3 , on 1759 batches
first_index : 384 and last_idx : 512
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
end of batch n°: 3, time: 1.0153799057006836
starting batch n° 4 , on 1759 batches
first_index : 512 and last_idx : 640
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
end of batch n°: 4, time: 1.0600237846374512
starting batch 

In [8]:
# renumber the index column (not accurate bc origin from each concatenation)

stopwords_lemmas_df = model_lemmas_df.reset_index()
stopwords_postag_df = model_postag_df.reset_index()

# drop the secoond index column

stopwords_lemmas_df = stopwords_lemmas_df.drop(labels = "index", axis=1)
stopwords_postag_df = stopwords_postag_df.drop(labels = "index", axis=1)

# add these stopwords_lemmas_df and  stopwords_postag_df to the global df
df["lemmas_only_VERB_ADJ_ADV_NOUN"] = stopwords_lemmas_df
df["postags_only_VERB_ADJ_ADV_NOUN"] = stopwords_postag_df


In [3]:
# exporter le dataframe dans un csv

df.to_csv("df_CC_lemmas_postag.csv", index = False)

NameError: name 'df' is not defined

In [4]:
df = pd.read_csv("df_CC_lemmas_postag.csv", sep = ",", encoding = "utf-8", dtype= str)


In [21]:
# delete the stopwords, using the spacy lexicon

from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]
final_stopwords_set = set(final_stopwords_list)
delete_stopwords = lambda lemma_list : [x for x in lemma_list if x not in final_stopwords_set]


# add this df to the global df
# df["lemmas_only_VERB_ADJ_ADV_NOUN_without_stopwords"] = df["lemmas_only_VERB_ADJ_ADV_NOUN"].apply(delete_stopwords)
# df["all_lemmas_without_stopwords"] = df["lemmas"].apply(delete_stopwords)

95756     [u, c, u, n,  , g, e, s, t, e,  , e, n,  , f, ...
149801    [p, l, u, s,  , 1, 0,  , %,  , d, e, s,  , é, ...
116530    [1, 6, /, 0, 1, /, 2, 0, 1, 9, 
, B, n, j, u, ...
106247    [é, c, n, m, e,  , s, u, p, p, r, m, t,  , l, ...
106050    [[, I, N, T, E, R, N, E, T, ],  , T, R, :,  , ...
221292    [L, e,  , G, r, n, d,  , D, é, b, t,  , n, t, ...
142809    [D, é, m, c, r, t, e, e, t,  , c, t, e, n, n, ...
205818    [-, P, l, f, n, n, e, m, e, n, t,  , d, e, s, ...
83877     [P, u, r,  , e, n,  , d, n, n, e,  , q, u,  , ...
102508    [c,  , d, e, s, s, u, s,  , q, u, e, l, q, u, ...
Name: Contribution, dtype: object

In [20]:
df = df.sample(10)
# test_stpw = lambda lemma_list : [x for x in lemma_list if x not in final_stopwords_set]
truc = []
df_vide = pd.DataFrame(columns=["resultats"])
for values in df.itertuples(index=False) :
    print(values.lemmas)
    for word in list(values.lemmas) :
        print(word)
        if word not in final_stopwords_set :
            truc.append(word)
df_vide["resultats"] = [truc]
df_vide

['aucun', 'geste', 'en', 'faveur', 'de', 'retraité', 'percevoir', 'un', 'retraite', 'inférieur', 'à', '1200', 'euro', 'accorder', 'un', 'retraite', 'minimal', 'de', 'un', 'montant', 'de', '1200', 'euro', 'augmentation', 'de', 'minimum', 'vieillesse', 'même', 'base', 'de', 'calcul', 'pour', 'le', 'retraite', 'privé', 'et', 'public', 'assistant', 'maternel', 'maintien', 'de', 'droit', 'de', 'location', 'en', 'cas', 'de', 'perte', 'de', 'contrat', 'stop', 'à', 'tentation', 'intempestif', 'de', 'taxe', 'carburant', 'et', 'autre', 'réduction', 'de', 'nombre', 'de', 'élu', 'député', 'européen', 'député', 'national', 'sénateur', 'conseiller', 'régional', 'président', 'de', 'communauté', 'de', 'commune', 'maire', 'réduction', 'de', 'instance', 'région', 'état', 'département', 'canton', 'communauté', 'de', 'commune', 'commun', 'réduire', 'le', 'indemnité', 'de', 'élu', 'trouver', 'un', 'solution', 'moins', 'onéreux', 'pour', 'éviter', 'un', 'investissement', 'trop', 'coûteux', 'et', 'non', 'ren

,resultats
0,"[[, ', u, c, u, n, ', ,, , ', g, e, s, t, e, ..."


In [15]:
print_full(df["lemmas"])

116530                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      ['16', '01', '2019', '.', '  ', '.', 'je', 'souhaiter', 'un', 'meilleur', 'justice', 'social', 'fiscal', 'po

In [22]:
# test = [{"frite" : [["j'ai", "mangé", "des", "chips"], ["j'ai", "mangé", "des", "frites"], ["du", "les", "mais", "et", "ce"]]}]
sss = pd.DataFrame(data = {"frite" : [["j'ai", "mangé", "des", "chips"], ["j'ai", "mangé", "des", "frites"], ["du", "les", "mais", "et", "ce"]]})
print(sss)
sss["frite"].apply(delete_stopwords)

                        frite
0   [j'ai, mangé, des, chips]
1  [j'ai, mangé, des, frites]
2     [du, les, mais, et, ce]


0     [j'ai, mangé, chips]
1    [j'ai, mangé, frites]
2                       []
Name: frite, dtype: object

In [15]:
df["lemmas_only_VERB_ADJ_ADV_NOUN_without_stopwords"]

155104    [[, ', m, r, e, ', ,,  , ', v, r, e, ', ,,  , ...
219692    [[, ', d, é, m, c, r, t, e, ', ,,  , ', é, l, ...
116178    [[, ', n, m, ', ,,  , ', b, s, e, r, v, t, n, ...
126767    [[, ', h, r, m, n, s, t, n, ', ,,  , ', s, l, ...
48049     [[, ', g, r, u, p, e, ', ,,  , ', t, r, v, l, ...
75484     [[, ', e, x, m, n, e, r, ', ,,  , ', g, r, n, ...
55832     [[, ', d, l, é, n, c, e, ', ,,  , ', s, n, t, ...
70183     [[, ', d, e, u, x, è, m, e, ', ,,  , ', s, u, ...
114079    [[, ', v, t, e, ', ,,  , ', l, m, t, e, r, ', ...
49755     [[, ', e, x, e, m, p, l, e, ', ,,  , ', m, p, ...
Name: lemmas_only_VERB_ADJ_ADV_NOUN_without_stopwords, dtype: object

In [7]:
# exporter le dataframe dans un csv

df.to_csv("df_CC_lemmas_postag.csv", index = False)

: 